In [1]:
#importing the libraries
import numpy as np
import pandas as pd

#For Data preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer
ls= WordNetLemmatizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pg21p\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Loading the dataset
df=pd.read_csv('allery-V1.csv')
df.head()

,query,disease,context
0,What is allergic rhinitis ?,allergic rhintis,0.0
1,What are the signs and symptoms of allergic rh...,allergic rhintis,2.0
2,What are the complications of allergic rhinitis ?,allergic rhintis,2.0
3,What are the nasal symptoms of allergic rhinit...,allergic rhintis,2.0
4,"What are the ear, eye and throat symptoms of a...",allergic rhintis,2.0


In [3]:
#Taking the disease "allergic rhintis" and considering it's all queries refer to user 1 and storing in a new dataset
df_new=df[df.disease=='allergic rhintis']
df_new.shape

(112, 3)

In [2]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
prompt = "What is allergic rhinitis ?"
next_sentence = "What are the complications of allergic rhinitis ?"

encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

outputs = model(**encoding, labels=torch.LongTensor([1]))
logits = outputs.logits



In [5]:
outputs.loss

tensor(12.7992, grad_fn=<NllLossBackward>)

1. use NSP to predic next most probable query
2. this gives us the context of the query as well
3. use naive bayes to find context?

http://ceur-ws.org/Vol-2774/paper-02.pdf

### How the recommendation  system works for a single user

#### The dataset structure
| User query  | Context     | user_id | next_best_context |
| ----------- | ----------- | --------|-------------------|
| query1      | 0/1/2       |   1     |     2             |
| query2      | 0/1/2       |   2     |     3             |

Here the query is simply the user query, context can be of three types: Cause of disease/allergy(0), medication/treatment of disease/allergy(1), symptom of disease/allergy(2). Since a mdeical query can belong to any one of the contexts. 

#### How the recommendation takes place
1. Take user's input query ```q1```
2. Convert the query into vector  using tf-idf vectorisation technique
3. Try to predict the context of query using text classification methods(random forest, gradient boost) or Bert classification
4. Now suggest user the next best query belonging to the same context using text matching algorithms(cosine similarity, neural net)

In above method, we can try the steps 2,3 with both simple and advanced methods. Thus we can follow the two approaches.

In [6]:
print("new approach")

new approach


In [16]:
import tensorflow as tf
tf.__version__


'2.3.2'

In [11]:
!pip install bert-for-tf2
!pip install sentencepiece

  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30538 sha256=6bf546e0e86b92d7eddf0f1d04b34a1f161cf50d8353c887a26979dde801bef1
  Stored in directory: c:\users\pg21p\appdata\local\pip\cache\wheels\47\b6\e5\8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7915 sha256=169f89c7c8d722b13af4e25b11541827f9733663d3af1228bde4db141fc7faba
  Stored in directory: c:\users\pg21p\appdata\local\pip\cache\wheels\e1\11\67\33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19477 sha256=88c5e4df01f909c68834d26ba22765bf95142fc7f18186723116f1a0f44c8b63
  Stored in directory: c:\users\pg21p\appdata\local\pip\cache\wheels\0e\fc\d2\a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
Successfully built bert-for-tf2 py-params params-flow


# new approach

#### The dataset structure
| User query  | Context     | user_id | next_best_context |
| ----------- | ----------- | --------|-------------------|
| query1      | 0/1/2       |   1     |     2             |
| query2      | 0/1/2       |   2     |     3             |

## How it will work
1. Train the model with existing dataset
2. For a new query, try predicting its next context using the trained model.
3. Now when we get a context, we try NSP to find next most probable query of that particular context and display it.
4. Now same happens for upcoming query. but we already now have 2 queries into user history now, how to take account for them? may be we can make up a context probability table, and everytime a new query with specific context is suggested, we increase the count(and corresponding probability/log likelihood) of that context and take it into account from next time.

In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [2]:
#importing the libraries
import numpy as np
import pandas as pd

#For Data preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer
ls= WordNetLemmatizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pg21p\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Loading the dataset
df=pd.read_csv('allergy-V1.csv')
df.head()

,query,disease,context,next_best_context
0,What is allergic rhinitis ?,allergic rhintis,0.0,2.0
1,What are the signs and symptoms of allergic rh...,allergic rhintis,2.0,1.0
2,What are the complications of allergic rhinitis ?,allergic rhintis,2.0,0.0
3,What are the nasal symptoms of allergic rhinit...,allergic rhintis,2.0,2.0
4,"What are the ear, eye and throat symptoms of a...",allergic rhintis,2.0,2.0


In [4]:
#Taking the disease "allergic rhintis" and considering it's all queries refer to user 1 and storing in a new dataset
df_new=df[df.disease=='allergic rhintis']
df_new.shape

(112, 4)

In [5]:
df_new

,query,disease,context,next_best_context
0,What is allergic rhinitis ?,allergic rhintis,0.0,2.0
1,What are the signs and symptoms of allergic rh...,allergic rhintis,2.0,1.0
2,What are the complications of allergic rhinitis ?,allergic rhintis,2.0,0.0
3,What are the nasal symptoms of allergic rhinit...,allergic rhintis,2.0,2.0
4,"What are the ear, eye and throat symptoms of a...",allergic rhintis,2.0,2.0
...,...,...,...,...
107,Which medications in the drug class Nasal cort...,allergic rhintis,1.0,2.0
108,Which medications in the drug class Antihistam...,allergic rhintis,1.0,2.0
109,Which medications in the drug class Mast Cell ...,allergic rhintis,1.0,0.0
110,Which medications in the drug class Intranasal...,allergic rhintis,1.0,2.0


In [6]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

ImportError: cannot import name 'run_classifier' from 'bert' (C:\Users\pg21p\Anaconda3\envs\tensorflow\lib\site-packages\bert\__init__.py)

In [6]:
for i in range(0,112):
    if(df_new['next_best_context'][i]==0):
        df_new['next_best_context'][i]=3
df_new['next_best_context']

C:\Users\pg21p\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\pg21p\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


0      2.0
1      1.0
2      3.0
3      2.0
4      2.0
      ... 
107    2.0
108    2.0
109    3.0
110    2.0
111    3.0
Name: next_best_context, Length: 112, dtype: float64

In [8]:
train, val =  train_test_split(df_new, test_size = 0.2, random_state = 100)

In [9]:
DATA_COLUMN = 'query'
LABEL_COLUMN = 'next_best_context'
label_list = [1, 2, 3]

In [12]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

AttributeError: module 'bert' has no attribute 'run_classifier'

In [56]:
med_queries=df_new['query']

In [57]:
med_context=df_new['next_best_context']

In [60]:
BertTokenizer = bert.bert_tokenization.FullTokenizer #making tokeniser object
#bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            #trainable=False) #downloading the embedding layer of bert model
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() #saving as vocablary
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy() 
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [61]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [62]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [63]:
def tokenize_queries(medical_query):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(medical_query))

tokenized_queries = [tokenize_queries(query) for query in df_new['query']]

In [64]:
len(tokenized_queries)

112

In [65]:
queries_with_len = [[med_query, med_context[i], len(med_query)]
                 for i, med_query in enumerate(tokenized_queries)]

In [66]:
queries_with_len 

[[[2054, 2003, 27395, 1054, 20535, 7315, 1029], 2.0, 7],
 [[2054, 2024, 1996, 5751, 1998, 8030, 1997, 27395, 1054, 20535, 7315, 1029],
  1.0,
  12],
 [[2054, 2024, 1996, 12763, 1997, 27395, 1054, 20535, 7315, 1029], 3.0, 10],
 [[2054, 2024, 1996, 19077, 8030, 1997, 27395, 1054, 20535, 7315, 1029],
  2.0,
  11],
 [[2054,
   2024,
   1996,
   4540,
   1010,
   3239,
   1998,
   3759,
   8030,
   1997,
   27395,
   1054,
   20535,
   7315,
   1029],
  2.0,
  15],
 [[2029,
   6845,
   5852,
   2024,
   2109,
   1999,
   1996,
   11616,
   1997,
   27395,
   1054,
   20535,
   7315,
   1029],
  2.0,
  14],
 [[2129,
   2024,
   12126,
   2913,
   2109,
   1999,
   1996,
   11616,
   1998,
   9312,
   1997,
   27395,
   1054,
   20535,
   7315,
   1029],
  3.0,
  16],
 [[2054, 2024, 1996, 2350, 3949, 9942, 2005, 27395, 1054, 20535, 7315, 1029],
  1.0,
  12],
 [[2054, 2003, 1996, 3171, 10859, 1997, 27395, 1054, 20535, 7315, 1029],
  1.0,
  11],
 [[2054,
   2003,
   1996,
   4130,
   7361,
   1

In [67]:
queries_with_len.sort(key=lambda x: x[2])

In [68]:
sorted_queries_labels = [(query_lab[0], query_lab[1]) for query_lab in queries_with_len]

In [ ]:
sorted_queries_labels

In [70]:
#converting into a dataset that can be used by tensorflow
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_queries_labels, output_types=(tf.int32, tf.int32))

In [71]:
BATCH_SIZE = 20
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [72]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(20, 12), dtype=int32, numpy=
 array([[ 2054,  2003, 27395,  1054, 20535,  7315,  1029,     0,     0,
             0,     0,     0],
        [ 2129,  2003, 27395,  1054, 20535,  7315,  6219,  1029,     0,
             0,     0,     0],
        [ 2054,  5320, 24590, 27395,  1054, 20535,  7315,  1029,     0,
             0,     0,     0],
        [ 2054,  2003, 16928, 27395,  1054, 20535,  7315,  1029,     0,
             0,     0,     0],
        [ 2054,  2024,  8030,  1997, 27395,  1054, 20535,  7315,  1029,
             0,     0,     0],
        [ 2054,  2024, 12763,  1997, 27395,  1054, 20535,  7315,  1029,
             0,     0,     0],
        [ 2054,  2024,  1996, 12763,  1997, 27395,  1054, 20535,  7315,
          1029,     0,     0],
        [ 2054,  2024, 22575,  3896,  1997, 27395,  1054, 20535,  7315,
          1029,     0,     0],
        [ 2029, 19077, 11360,  9556,  6592, 27395,  1054, 20535,  7315,
          1029,     0,     0],
        [ 2054,  2024,  

In [73]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_queries_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [74]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=3,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [79]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 4

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [80]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [81]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="categorical_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [82]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
6/6 [==============================] 1.3947 - sparse_categorical_accuracy: 0.350 - 0s 48ms/step - loss: 1.3507 - sparse_categorical_accuracy: 0.425 - 0s 65ms/step - loss: 1.3425 - sparse_categorical_accuracy: 0.350 - 0s 75ms/step - loss: 1.3471 - sparse_categorical_accuracy: 0.312 - 0s 79ms/step - loss: 1.3432 - sparse_categorical_accuracy: 0.290 - 1s 84ms/step - loss: 1.3405 - sparse_categorical_accuracy: 0.276 - 1s 96ms/step - loss: 1.3405 - sparse_categorical_accuracy: 0.2768
Epoch 2/5
6/6 [==============================] - ETA: 0s - loss: 1.1622 - sparse_categorical_accuracy: 0.350 - ETA: 0s - loss: 1.1535 - sparse_categorical_accuracy: 0.325 - ETA: 0s - loss: 1.1324 - sparse_categorical_accuracy: 0.383 - ETA: 0s - loss: 1.1221 - sparse_categorical_accuracy: 0.387 - ETA: 0s - loss: 1.1104 - sparse_categorical_accuracy: 0.410 - ETA: 0s - loss: 1.0957 - sparse_categorical_accuracy: 0.419 - 1s 87ms/step - loss: 1.0957 - sparse_categorical_accuracy: 0.4196
Epoch 3/5
6/6 [====

In [ ]:
next(iter(test_dataset))

In [ ]:
results = text_model.evaluate(test_data)
print(results)